# Keep pulling same bandit as long as reward keeps coming!

In [1]:
!pip install kaggle-environments --upgrade

     |████████████████████████████████| 107 kB 420 kB/s 
  Attempting uninstall: kaggle-environments
    Found existing installation: kaggle-environments 1.7.0
    Uninstalling kaggle-environments-1.7.0:
      Successfully uninstalled kaggle-environments-1.7.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# observation = {
#     'remainingOverageTime': 60,
#     'agentIndex': 1, # 0 or 1
#     'reward': 92, # total reward
#     'step': 184, # [0-1999]
#     'lastActions': [84, 94]
# }

# configuration:
# {'episodeSteps': 2000,
#  'actTimeout': 0.25,
#  'runTimeout': 1200,
#  'banditCount': 100,
#  'decayRate': 0.97,
#  'sampleResolution': 100}

In [3]:
from kaggle_environments import make
env = make("mab", debug=True)

Loading environment football failed: No module named 'gfootball'


In [4]:
%%writefile mybest.py
import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict
from scipy.stats import beta

# random number generator
rng = np.random.default_rng(13)

# belief distribution for the thresholds of each arm
time_since_op_pull = np.zeros(100)
action_counts = np.zeros((100, 2), np.int16)
supports = np.tile(np.arange(101.0), (100, 1))
beliefs = 1e6 + beta(2,2).pdf(np.linspace(0,1,101))
beliefs = np.full((100, 101), beliefs/beliefs.sum())

# game history
step = 0
my_pulls = np.array([], dtype=np.int)
op_pulls = np.array([], dtype=np.int)
results = np.array([], dtype=np.int)


total_reward = 0
bandit_dict = {}

def set_seed(my_seed=42):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit():
    best_bandit = 0
    best_bandit_expected = 0
    for bnd in bandit_dict:
        expect = (bandit_dict[bnd]['win'] - bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'] - (bandit_dict[bnd]['opp']>0)*1.5) \
                 / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit

my_action_list = []
op_action_list = []
pre_wins = []


op_continue_cnt_dict = defaultdict(int)

# could add opposite tilting to those only pulled once?
def sample():
    window = 10
    hist = np.bincount(op_pulls[-window:], minlength=100).reshape(100, 1)
    hist = np.where(hist >= 2.0, hist, 0.0)

    tilted = (np.ceil(supports) ** hist) * beliefs
    tilted = tilted / tilted.sum(axis=1, keepdims=True)

    hist = np.bincount(op_pulls[-101:], minlength=100).reshape(100, 1)
    hist = np.where(hist == 1, 1.0, 0.0)
    hist[op_pulls[-1]] = 0.0

    tilted = ((101.0 - np.ceil(supports)) ** hist) * tilted
    tilted = tilted / tilted.sum(axis=1, keepdims=True)

    # get optimistic estimate of threshold
    optimism = 1.0
    mean = np.sum(supports * tilted, axis=1, keepdims=True)
    var = np.sum(((supports - mean) ** 2) * tilted, axis=1, keepdims=True)
    estimate = np.squeeze(mean + optimism * np.sqrt(var))
    
    # choose the arm with highest estimate
    maximizers = np.flatnonzero(estimate == estimate.max())
    return rng.choice(maximizers)


# you can probably infer opponent beliefs one step behind depending on whether
# they sample again... so if they pull, pull again they won last time, if they
# pull, move, the lost etc. if they pull, pull, move it was one-one...

# keep opponent beliefs and the estimate weighted average of our beliefs
# and theirs...

def update():
    global beliefs, supports, action_counts, time_since_op_pull

    my_pull = my_pulls[-1]
    op_pull = op_pulls[-1]
    result = results[-1]

    # bayesian update for the result of our pull
    likelihood = np.ceil(supports[my_pull, :])
    likelihood = (101.0 - likelihood) if result == 0 else likelihood
    posterior = likelihood * beliefs[my_pull, :]
    beliefs[my_pull, :] = posterior / posterior.sum()

    # if the opponent repeats a first-time action, assume the first time is a success
    if (action_counts[op_pull,1] == 2) and (op_pulls[-2] == op_pull):
        likelihood = np.ceil(supports[op_pull, :])
        posterior = likelihood * beliefs[op_pull, :]
        beliefs[op_pull, :] = posterior / posterior.sum()    

    # if the opponent hasn't pulled a lever in a long time then
    # it is probably because the first time was a failure
    for pull in np.where((action_counts[:,1] == 1) & (time_since_op_pull > 100)):
        likelihood = 101 - np.ceil(supports[pull, :])
        posterior = likelihood * beliefs[pull, :]
        beliefs[pull, :] = posterior / posterior.sum()
        time_since_op_pull[pull] = -np.inf

    # decay in threshold due to pull
    supports[my_pull, :] = 0.97 * supports[my_pull, :]

    # decay due to opponent's pull
    #supports[op_pull, :] = 0.97 * supports[op_pull, :]

    # increment counts
    action_counts[my_pull,0] += 1
    action_counts[op_pull,1] += 1

    # update time since pull
    time_since_op_pull[op_pull] = 0
    time_since_op_pull += 1
    return

# random number generator
rng = np.random.default_rng(13)

# belief distribution for the thresholds of each arm
supports = np.tile(np.arange(101.0), (100, 1))
beliefs = np.full_like(supports, 1 / 101.0)

# game history
step = 0
my_pulls = np.array([], dtype=np.int)
op_pulls = np.array([], dtype=np.int)
results = np.array([], dtype=np.int)

# the main function called by kaggle environment for each turn
def agent(observation, configuration):
    global total_reward, bandit_dict,conti
    global step, my_pulls, op_pulls, results

    my_pull = random.randrange(configuration['banditCount'])
    if observation['step'] == 0:
        total_reward = 0
        bandit_dict = {}
        conti=0
        for x in range(configuration['banditCount']):
            bandit_dict[x] = {'win': 1, 'loss': 0, 'opp': 0, 'my_continue': 0, 'op_continue': 0}
            
        return int(rng.integers(0, 100))  
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        pre_wins.append(last_reward)
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        if last_reward > 0:
            bandit_dict[my_last_action]['win'] += 1
        else:
            bandit_dict[my_last_action]['loss'] += 1
        bandit_dict[op_last_action]['opp'] += 1

    my_pull = observation.lastActions[observation.agentIndex]
    op_pull = observation.lastActions[1 - observation.agentIndex]
    result = observation.reward - results.sum()

    # update game history
    my_pulls = np.append(my_pulls, my_pull)
    op_pulls = np.append(op_pulls, op_pull)
    results = np.append(results, result)
    update()
    if(last_reward>0):
        my_pull=my_last_action
        conti+=1
        if (step>0 and step<200)or((step>600 and step<900))and((step>1700 and step<2000)):
            return my_pull
    if step>3 and (len(set(my_action_list[-4:]))==1)and(sum(results[-5:])>1):
        my_pull = my_action_list[-1]
        conti=0
        if (step>700 and step<900):
            return my_pull
    if step>4 and (len(set(op_action_list[-3:]))==1) and step<600:
         my_pull = op_action_list[-1]
         conti=0   
         return   my_pull
           
    if (step>700 and step<900):
        conti=0
        my_pull = get_next_bandit()
        return my_pull
    else:
        my_pull =int(sample()) 
        conti=0
        return my_pull


Writing mybest.py
